<a href="https://colab.research.google.com/github/iroliak/ekpa_1/blob/main/IPS_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [2]:
# Load data
url = 'https://raw.githubusercontent.com/kdemertzis/EKPA/main/Data/pcap_data.csv'
data = pd.read_csv(url)

In [3]:
# Preprocessing
X = data.drop('target', axis=1)
y = data['target']
y = to_categorical(y)  # One-hot encoding

In [4]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [5]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [6]:
# Build the model
def build_model(input_dim, output_dim):
    model = Sequential([
        Dense(128, input_dim=input_dim, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(output_dim, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model(X_train.shape[1], y_train.shape[1])

In [7]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_split=0.2, epochs=100, callbacks=[early_stopping], batch_size=32)

Epoch 1/100
300/300 [==============================] - 4s 6ms/step - loss: 0.7524 - accuracy: 0.6869 - val_loss: 0.4674 - val_accuracy: 0.8630
Epoch 2/100
300/300 [==============================] - 1s 3ms/step - loss: 0.5493 - accuracy: 0.8172 - val_loss: 0.4113 - val_accuracy: 0.8856
Epoch 3/100
300/300 [==============================] - 1s 3ms/step - loss: 0.5059 - accuracy: 0.8523 - val_loss: 0.3908 - val_accuracy: 0.8998
Epoch 4/100
300/300 [==============================] - 1s 3ms/step - loss: 0.4784 - accuracy: 0.8630 - val_loss: 0.3784 - val_accuracy: 0.9040
Epoch 5/100
300/300 [==============================] - 1s 3ms/step - loss: 0.4611 - accuracy: 0.8761 - val_loss: 0.3642 - val_accuracy: 0.9098
Epoch 6/100
300/300 [==============================] - 1s 3ms/step - loss: 0.4465 - accuracy: 0.8801 - val_loss: 0.3594 - val_accuracy: 0.9090
Epoch 7/100
300/300 [==============================] - 1s 3ms/step - loss: 0.4395 - accuracy: 0.8853 - val_loss: 0.3494 - val_accuracy: 0.9115

In [8]:
# Evaluate the model
evaluation = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {evaluation[1]*100:.2f}%")

94/94 [==============================] - 0s 2ms/step - loss: 0.3508 - accuracy: 0.9165
Test Accuracy: 91.65%


In [9]:
# Define the action to be taken based on predictions
def take_action(predictions):
    threshold = 0.5  # Set a threshold for classification probability
    actions = []
    for i, prediction in enumerate(predictions):
        # Determine if any class probability exceeds the threshold
        if np.max(prediction) > threshold:
            class_id = np.argmax(prediction)
            if class_id == 1:  # Assuming class '1' is malicious
                actions.append(f"Alert: Action required for index {i}, potential threat detected.")
            else:
                actions.append(f"Normal: No action needed for index {i}.")
        else:
            actions.append(f"Uncertain: Further investigation needed for index {i}.")
    return actions

In [10]:
# Predict and take action
predictions = model.predict(X_test)
actions = take_action(predictions)

94/94 [==============================] - 0s 2ms/step


In [11]:
# Print actions
for action in actions:
    print(action)

Normal: No action needed for index 0.
Normal: No action needed for index 1.
Normal: No action needed for index 2.
Normal: No action needed for index 3.
Normal: No action needed for index 4.
Normal: No action needed for index 5.
Normal: No action needed for index 6.
Normal: No action needed for index 7.
Normal: No action needed for index 8.
Normal: No action needed for index 9.
Normal: No action needed for index 10.
Normal: No action needed for index 11.
Normal: No action needed for index 12.
Normal: No action needed for index 13.
Normal: No action needed for index 14.
Normal: No action needed for index 15.
Normal: No action needed for index 16.
Normal: No action needed for index 17.
Normal: No action needed for index 18.
Alert: Action required for index 19, potential threat detected.
Normal: No action needed for index 20.
Normal: No action needed for index 21.
Normal: No action needed for index 22.
Normal: No action needed for index 23.
Normal: No action needed for index 24.
Normal: No

Δημιουργήθηκε μια συνάρτηση take_action που λαμβάνει ως είσοδο τις προβλέψεις του μοντέλου και αναλόγως αποφασίζει για την ανάληψη ενεργειών βάσει καθορισμένου κατωφλίου πιθανότητας.
Για κάθε πρόβλεψη που υπερβαίνει το κατώφλι, ελέγχεται αν η πρόβλεψη αντιστοιχεί σε επιθετική συμπεριφορά και λαμβάνονται τα αντίστοιχα μέτρα.